In [2]:
import os
import nibabel as nib
import numpy as np
from PIL import Image
from tqdm import tqdm

def nii_to_png(img_dir, label_dir, output_img_dir, output_mask_dir):
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)

    img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])
    label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.nii') or f.endswith('.nii.gz')])

    assert len(img_files) == len(label_files), "图像和标签数量不一致"

    idx = 0  # 用于文件命名编号

    for img_file, label_file in tqdm(zip(img_files, label_files), total=len(img_files)):
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, label_file)

        # 读取 nii 文件
        img_nii = nib.load(img_path).get_fdata()
        label_nii = nib.load(label_path).get_fdata()

        # 检查维度一致
        assert img_nii.shape == label_nii.shape, f"维度不一致: {img_file}"

        # 对每一层切片进行保存（Z 轴）
        for i in range(img_nii.shape[2]):
            img_slice = img_nii[:, :, i]
            label_slice = label_nii[:, :, i]

            # Normalize 图像到 0~255（标签假设已为 0/1）
            img_norm = ((img_slice - np.min(img_slice)) / (np.ptp(img_slice) + 1e-8) * 255).astype(np.uint8)
            label_bin = ((label_slice > 0) * 255).astype(np.uint8)  # 处理为二值掩码

            img_pil = Image.fromarray(img_norm).convert('L')  # 单通道灰度图
            label_pil = Image.fromarray(label_bin).convert('L')

            img_pil.save(os.path.join(output_img_dir, f"image_{idx:05d}.png"))
            label_pil.save(os.path.join(output_mask_dir, f"mask_{idx:05d}.png"))

            idx += 1

if __name__ == "__main__":
    # 输入路径
    img_dir = "/home/hxy/Desktop/medAI_template/raw_datasets/RawData/Training/img"
    label_dir = "/home/hxy/Desktop/medAI_template/raw_datasets/RawData/Training/label"

    # 输出路径
    output_img_dir = "./processed_dataset/images/"
    output_mask_dir = "./processed_dataset/masks/"

    nii_to_png(img_dir, label_dir, output_img_dir, output_mask_dir)


100%|██████████| 30/30 [02:04<00:00,  4.16s/it]
